In [1]:
import boto3
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[01/22/25 04:49:30] INFO     Found credentials from IAM Role:                                   ]8;id=40413;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=886772;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
s3=boto3.client('s3')
sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'shalu97-project'
print("Using bucket " + bucket)

[01/22/25 04:49:42] INFO     Found credentials from IAM Role:                                   ]8;id=483723;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=658634;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Using bucket shalu97-project


In [3]:
datas=pd.read_csv('loan_data.csv')
datas

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,27.0,male,Associate,47971.0,6,RENT,15000.0,MEDICAL,15.66,0.31,3.0,645,No,1
44996,37.0,female,Associate,65800.0,17,RENT,9000.0,HOMEIMPROVEMENT,14.07,0.14,11.0,621,No,1
44997,33.0,male,Associate,56942.0,7,RENT,2771.0,DEBTCONSOLIDATION,10.02,0.05,10.0,668,No,1
44998,29.0,male,Bachelor,33164.0,4,RENT,12000.0,EDUCATION,13.23,0.36,6.0,604,No,1


In [4]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      45000 non-null  float64
 1   person_gender                   45000 non-null  object 
 2   person_education                45000 non-null  object 
 3   person_income                   45000 non-null  float64
 4   person_emp_exp                  45000 non-null  int64  
 5   person_home_ownership           45000 non-null  object 
 6   loan_amnt                       45000 non-null  float64
 7   loan_intent                     45000 non-null  object 
 8   loan_int_rate                   45000 non-null  float64
 9   loan_percent_income             45000 non-null  float64
 10  cb_person_cred_hist_length      45000 non-null  float64
 11  credit_score                    45000 non-null  int64  
 12  previous_loan_defaults_on_file  

In [5]:
datas.isna().sum()

person_age                        0
person_gender                     0
person_education                  0
person_income                     0
person_emp_exp                    0
person_home_ownership             0
loan_amnt                         0
loan_intent                       0
loan_int_rate                     0
loan_percent_income               0
cb_person_cred_hist_length        0
credit_score                      0
previous_loan_defaults_on_file    0
loan_status                       0
dtype: int64

In [6]:
datas.drop(['person_gender','person_education','person_home_ownership'],axis=1,inplace=True,errors='ignore')
datas

,person_age,person_income,person_emp_exp,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,71948.0,0,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,12282.0,0,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,12438.0,3,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,79753.0,0,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,66135.0,1,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1
...,...,...,...,...,...,...,...,...,...,...,...
44995,27.0,47971.0,6,15000.0,MEDICAL,15.66,0.31,3.0,645,No,1
44996,37.0,65800.0,17,9000.0,HOMEIMPROVEMENT,14.07,0.14,11.0,621,No,1
44997,33.0,56942.0,7,2771.0,DEBTCONSOLIDATION,10.02,0.05,10.0,668,No,1
44998,29.0,33164.0,4,12000.0,EDUCATION,13.23,0.36,6.0,604,No,1


In [7]:
features=list(datas.columns)
features

['person_age',
 'person_income',
 'person_emp_exp',
 'loan_amnt',
 'loan_intent',
 'loan_int_rate',
 'loan_percent_income',
 'cb_person_cred_hist_length',
 'credit_score',
 'previous_loan_defaults_on_file',
 'loan_status']

In [8]:
labels=features.pop(-1)
labels

'loan_status'

In [9]:
label_encoder={}
for i in['loan_intent','previous_loan_defaults_on_file']:
    label_encoder[i]=LabelEncoder()
    datas[i]=label_encoder[i].fit_transform(datas[i])

In [10]:
x=datas[features]
x

,person_age,person_income,person_emp_exp,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file
0,22.0,71948.0,0,35000.0,4,16.02,0.49,3.0,561,0
1,21.0,12282.0,0,1000.0,1,11.14,0.08,2.0,504,1
2,25.0,12438.0,3,5500.0,3,12.87,0.44,3.0,635,0
3,23.0,79753.0,0,35000.0,3,15.23,0.44,2.0,675,0
4,24.0,66135.0,1,35000.0,3,14.27,0.53,4.0,586,0
...,...,...,...,...,...,...,...,...,...,...
44995,27.0,47971.0,6,15000.0,3,15.66,0.31,3.0,645,0
44996,37.0,65800.0,17,9000.0,2,14.07,0.14,11.0,621,0
44997,33.0,56942.0,7,2771.0,0,10.02,0.05,10.0,668,0
44998,29.0,33164.0,4,12000.0,1,13.23,0.36,6.0,604,0


In [11]:
y=datas[labels]
y

0        1
1        0
2        1
3        1
4        1
        ..
44995    1
44996    1
44997    1
44998    1
44999    1
Name: loan_status, Length: 45000, dtype: int64

In [12]:
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [13]:
trainX = pd.DataFrame(X_train)
testX = pd.DataFrame(X_test)

trainX[labels] = y_train
testX[labels] = y_test


print(trainX.shape)
print(testX.shape)

(36000, 11)
(9000, 11)


In [14]:
trainX.to_csv("train.csv",index = False)
testX.to_csv("test.csv", index = False)

In [15]:
# send data to S3. SageMaker will take training data from s3
sk_prefix = "loan-data"
trainpath = sess.upload_data(
    path="train.csv", bucket=bucket, key_prefix=sk_prefix
)

testpath = sess.upload_data(
    path="test.csv", bucket=bucket, key_prefix=sk_prefix
)

In [16]:
trainpath

's3://shalu97-project/loan-data/train.csv'

In [17]:
testpath

's3://shalu97-project/loan-data/test.csv'

In [18]:
%%writefile script.py
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO 
import argparse
import joblib
import os
import numpy as np
import pandas as pd

# inference functions ---------------

def input_fn(request_body, request_content_type):
    print(request_body)
    print(request_content_type)
    if request_content_type == "text/csv":
        request_body = request_body.strip()
        try:
            df = pd.read_csv(StringIO(request_body), header=None)
            return df
        
        except Exception as e:
            print(e)
    else:
        return """Please use Content-Type = 'text/csv' and, send the request!!""" 
 
    
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

def predict_fn(input_data, model):
    if type(input_data) != str:
        prediction = model.predict(input_data)
        print(prediction)
        return prediction
    else:
        return input_data
        
    
if __name__ == "__main__":

    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    parser.add_argument("--n_estimators", type=int, default=100)
    parser.add_argument("--random_state", type=int, default=0)
    


    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="train.csv")
    parser.add_argument("--test-file", type=str, default="test.csv")

    args, _ = parser.parse_known_args()
    
    print("SKLearn Version: ", sklearn.__version__)
    print("Joblib Version: ", joblib.__version__)

    print("[INFO] Reading data")
    print()
    
    train_df = pd.read_csv(os.path.join(args.train,args.train_file))
    test_df = pd.read_csv(os.path.join(args.test,args.test_file))
    
    features = list(train_df.columns)
    labels = features.pop(-1)
    
    print("Building training and testing datasets")
    print()
    X_train = train_df[features]
    X_test = test_df[features]
    y_train = train_df[labels]
    y_test = test_df[labels]

    print('Column order: ')
    print(features)
    print()
    
    print("Label column is: ",labels)
    print()
    
    print("Data Shape: ")
    print()
    print("---- SHAPE OF TRAINING DATA (80%) ----")
    print(X_train.shape)
    print(y_train.shape)
    print()
    print("---- SHAPE OF TESTING DATA (20%) ----")
    print(X_test.shape)
    print(y_test.shape)
    print()
    
  
    print("Training RandomForest Model.....")
    print()
    model =  RandomForestClassifier(n_estimators=args.n_estimators, random_state=args.random_state, verbose = 2,n_jobs=-1)
    model.fit(X_train, y_train)
    print()
    

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("Model persisted at " + model_path)
    print()

    
    y_pred_test = model.predict(X_test)
    test_acc = accuracy_score(y_test,y_pred_test)
    test_rep = classification_report(y_test,y_pred_test)

    print()
    print("---- METRICS RESULTS FOR TESTING DATA ----")
    print()
    print("Total Rows are: ", X_test.shape[0])
    print('[TESTING] Model Accuracy is: ', test_acc)
    print('[TESTING] Testing Report: ')
    print(test_rep)

Overwriting script.py


In [19]:
! python script.py --n_estimators 100 \
                   --random_state 0 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \

[INFO] Extracting arguments
SKLearn Version:  1.5.2
Joblib Version:  1.4.2
[INFO] Reading data

Building training and testing datasets

Column order: 
['person_age', 'person_income', 'person_emp_exp', 'loan_amnt', 'loan_intent', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length', 'credit_score', 'previous_loan_defaults_on_file']

Label column is:  loan_status

Data Shape: 

---- SHAPE OF TRAINING DATA (80%) ----
(36000, 10)
(36000,)

---- SHAPE OF TESTING DATA (20%) ----
(9000, 10)
(9000,)

Training RandomForest Model.....

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 

In [20]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.m5.xlarge",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="Custom-sklearn",
    hyperparameters={
        "n_estimators": 100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait=7200,
    max_run=3600
)

In [21]:
# Launch the training job asynchronously with the 'wait=True' argument
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=True)

[01/22/25 04:53:18] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=493897;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=915480;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[01/22/25 04:53:19] INFO     Creating training-job with name:                                       ]8;id=515735;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=785456;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             Custom-sklearn-2025-01-22-04-53-18-950                                                

2025-01-22 04:53:19 Starting - Starting the training job...
2025-01-22 04:53:45 Starting - Preparing the instances for training...
2025-01-22 04:54:22 Downloading - Downloading the training image......
2025-01-22 04:55:13 Training - Training image download completed. Training in progress.
2025-01-22 04:55:13 Uploading - Uploading generated training model2025-01-22 04:55:07,726 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-01-22 04:55:07,728 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-01-22 04:55:07,766 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-01-22 04:55:07,902 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-01-22 04:55:07,914 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-01-22 04:55:07,925 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-

In [22]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)


2025-01-22 04:55:25 Starting - Preparing the instances for training
2025-01-22 04:55:25 Downloading - Downloading the training image
2025-01-22 04:55:25 Training - Training image download completed. Training in progress.
2025-01-22 04:55:25 Uploading - Uploading generated training model
2025-01-22 04:55:25 Completed - Training job completed
Model artifact persisted at s3://sagemaker-ap-south-1-850995560981/Custom-sklearn-2025-01-22-04-53-18-950/output/model.tar.gz


In [23]:
from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

# Generate a unique model name using timestamp
model_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
# Create the SKLearnModel object
model = SKLearnModel(
    name=model_name,
    model_data=artifact,                  # Path to the model artifact in S3
    role=get_execution_role(),            # Role that has permission to access resources like S3
    entry_point="script.py",              # Python script that will be used to serve the model
    framework_version=FRAMEWORK_VERSION,  
)

In [24]:
# code for deploy a model to a SageMaker endpoint
endpoint_name = "Custom-sklearn-model-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)

EndpointName=Custom-sklearn-model-2025-01-22-04-56-29


[01/22/25 04:56:29] INFO     Creating model with name: Custom-sklearn-model-2025-01-22-04-56-12     ]8;id=637933;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=121871;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[01/22/25 04:56:30] INFO     Creating endpoint-config with name                                     ]8;id=590660;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=313964;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             Custom-sklearn-model-2025-01-22-04-56-29                                              

                    INFO     Creating endpoint with name Custom-sklearn-model-2025-01-22-04-56-29   ]8;id=485717;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=539620;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

-----!

In [25]:
testX[features][0:3].values.tolist()

[[32.0, 96865.0, 10.0, 7500.0, 1.0, 6.04, 0.08, 10.0, 601.0, 0.0],
 [24.0, 56838.0, 6.0, 9000.0, 1.0, 11.49, 0.16, 4.0, 647.0, 1.0],
 [22.0, 37298.0, 0.0, 5000.0, 3.0, 14.88, 0.13, 4.0, 711.0, 0.0]]

In [26]:
print(predictor.predict(testX[features][0:2].values.tolist()))

Please use Content-Type = 'text/csv' and, send the request!!


In [27]:
# sm_boto3.delete_endpoint(EndpointName=predictor.endpoint)